# 8. Geography

Datashader supports any type of data that can be aggregated in two dimensions, but it also provides some specific support for geographic and Earth-science functionality.  See [GeoViews.org](http://geoviews.org) for a wide array of other geographic support designed to work well with Datashader.

Most of the Datashader geo-related functionality is part of the `datashader.geo` module:

* [Slope](#ds.geo---slope-function)
* [Aspect](#ds.geo---aspect-function)
* [Hillshade](#ds.geo---hillshade-function)
* [NDVI](#ds.geo---ndvi-function)
* [Mean](#ds.geo---mean-function)

In [ ]:
from datashader.geo import slope, aspect, hillshade, ndvi

In [ ]:
import datashader as ds
import xarray as xr

import holoviews as hv
hv.extension('matplotlib','bokeh')
COLORMAP = 'copper'

from IPython.display import display, Markdown

def version(mod):
    print('{} version: {}'.format(mod.__name__,mod.__version__))

version(ds)
version(xr)
version(hv)

In [ ]:
def helper_normalize(data_array):
    """
    Simple normalization helper
    """
    return (data_array - data_array.min()) / (data_array.max() - data_array.min())


def helper_do_gaussian():
    """Create a gaussian surface"""
    import numpy as np
    import xarray as xr

    X = Y = np.linspace(-4, 4, 301)
    X,Y = np.meshgrid(X, Y, sparse=True)
    X_fac = -np.power(X, 2)
    Y_fac = -np.power(Y, 2)

    data_gaussian = np.exp((X_fac+Y_fac))
    data_gaussian = helper_normalize(data_gaussian)

    return xr.DataArray(data_gaussian, attrs={'res':1})


def helper_do_gaussian_inverse():
    """Return a gaussian with inverted values"""
    data_gaussian = helper_do_gaussian()
    data_gaussian_inv = -1 * (data_gaussian - 1)
    return data_gaussian_inv

## Slope
* Burrough, P. A., and McDonell, R. A., 1998. *Principles of Geographical Information Systems* (Oxford University Press, New York), pp 406

[Slope](https://en.wikipedia.org/wiki/Slope) is the inclination of a surface. 
In geography, *slope* is amount of change in elevation of a terrain regarding its surroundings.

In [ ]:
display(Markdown(slope.__doc__))

In [ ]:
# We will use a gaussian surface to show the output of the `slope` function.

xarray_gaussian = helper_do_gaussian()

da_slope = slope(xarray_gaussian)

img_gauss = hv.Image(xarray_gaussian.data, label='Gaussian curve').options(colorbar=True).options(cmap=COLORMAP)
img_slope = hv.Image(da_slope.data, label='Slope output').options(colorbar=True).options(cmap=COLORMAP)
img_slope + img_gauss

## Data

The following section provides synthetic and actual geospatial data to demonstrate geoscience functionality.


#### Synthetic

* [Gaussian](#Gaussian) : a gaussian surface that will be used for the `geo.hillshade` function
 * Since a Gaussian distribution is simple, symmetric it is a good *toy* example for those 3D functions like shading or slope projections.
* [Random](#Random---Normal) : "white-noise" to be used in `geo.mean` function
 * The visual effect of the mean filter is to *blur* the image. Another important property of the mean filter is that is conserves the total, integrate intensity of the image. Both aspects are clearly visible from a uniform (random) distribution.
 
#### Real
* [Austin-DEM](#Austin---DEM) : image providing elevation for the region surrounding the city of Austin, TX, USA; we will use the Austin-DEM data to produce a shaded (`hillshade`) version of the image.
* [Landsat](#Landsat) : Landsat is a Earth observation mission -- currently in its 8th phase -- providing satellite images in different wavelengths. In particular, we will use two images of the same region but different waveband -- red and near-infrared -- to estimate the vegetation coverage from the `ndvi` method.

## Aspect
* Burrough, P. A., and McDonell, R. A., 1998. *Principles of Geographical Information Systems* (Oxford University Press, New York), pp 406

[Aspect](https://en.wikipedia.org/wiki/Aspect_(geography)) is the orientation of slope, measured clockwise in degrees from 0 to 360, where 0 is north-facing, 90 is east-facing, 180 is south-facing, and 270 is west-facing.

In [ ]:
display(Markdown(aspect.__doc__))

In [ ]:
# We will use a gaussian surface to help us understand the result of the *aspect* transform.

xarray_gaussian = helper_do_gaussian()

da_aspect = aspect(xarray_gaussian)

hv.Image(da_aspect.data, label='Aspect of Gaussian centered at [0,0]').options(colorbar=True).options(cmap=COLORMAP)

## Hillshade
* Burrough, P. A., and McDonell, R. A., 1998. *Principles of Geographical Information Systems* (Oxford University Press, New York), pp 406

[Hillshade](https://en.wikipedia.org/wiki/Terrain_cartography) is a technique used to visualize terrain as shaded relief, illuminating it with a hypothetical light source. The illumination value for each cell is determined by its orientation to the light source, which is based on slope and aspect.

In [ ]:
display(Markdown(hillshade.__doc__))

#### Multi-Directional Oblique (MDOW) Hillshade

In [ ]:
hillshade_agg = hillshade(xarray_gaussian, how='mdow', out_type='data')
img_shade = hv.Image(hillshade_agg.data, label='Shade (MDOW) from Gaussian')
img_shade.options(colorbar=True).options(cmap='gray') + img_gauss.options(cmap=COLORMAP)

## NDVI
* Burrough, P. A., and McDonell, R. A., 1998. *Principles of Geographical Information Systems* (Oxford University Press, New York), pp 406

The Normalized Difference Vegetation Index (NDVI) quantifies vegetation by measuring the difference between near-infrared (which vegetation strongly reflects) and red light (which vegetation absorbs).
NDVI always ranges from -1 to +1. But there isn’t a distinct boundary for each type of land cover.
For example, when you have negative values, it’s highly likely that it’s water. On the other hand, if you have a NDVI value close to +1, there’s a high possibility that it’s dense green leaves.
But when NDVI is close to zero, there isn’t green leaves and it could even be an urbanized area.

In [ ]:
display(Markdown(ndvi.__doc__))

In [ ]:
import datashader.transfer_functions as tf

NIR_data = helper_do_gaussian()
RED_data = helper_do_gaussian_inverse()

da_ndvi = ndvi(NIR_data, RED_data)
hv.Image(da_ndvi.data).options(colorbar=True).options(cmap='PRGn')

## Real data

Here we are going to see how those methods transform real images.
The images represent the region around Austin, TX in USA, which is mountainous region and make a good example for our `hillshade` method.
The other data set is composed by two images from the Landsat satellite, the *red* and *near-infrared* filters, which we will use for checking the `ndvi` processing effects.

## Austin terrain

```
import numpy as np
import rasterio
    
# Austin-DEM
austin_dem = '../data/austin_dem.tif'

elevation_data = rasterio.open(austin_dem)
elevation_xarray = xr.DataArray(elevation_data.read().astype(np.uint16))

xmin = elevation_data.bounds.left
ymin = elevation_data.bounds.bottom
xmax = elevation_data.bounds.right
ymax = elevation_data.bounds.top

w = np.ceil(elevation_data.width / 8.0)
h = np.ceil(elevation_data.height / 8.0)

cvs = ds.Canvas(plot_width=w, plot_height=h, 
                x_range=(xmin,xmax), y_range=(ymin,ymax))
agg = cvs.raster(elevation_xarray[0])
hillshade(agg, out_type='data')
```

```
da_elevation = xr.open_rasterio(austin_dem)

# Define Datashader Canvas
from datashader import Canvas
from math import ceil

# xmin = elevation_data.bounds.left
xmin = da_elevation.x.min()

# ymin = elevation_data.bounds.bottom
ymin = da_elevation.y.min()

# xmax = elevation_data.bounds.right
xmax = da_elevation.x.max()

# ymax = elevation_data.bounds.top
ymax = da_elevation.y.max()

# w = np.ceil(elevation_data.width / 8.0)
w = ceil(len(da_elevation.x) / 8.0)

# h = np.ceil(elevation_data.height / 8.0)
h = ceil(len(da_elevation.y) / 8.0)

cvs = Canvas(plot_width=w, plot_height=h, x_range=(xmin,xmax), y_range=(ymin,ymax))

xarr = da_elevation.load()[0]

# Rasterize our data according to the Canvas resolution/definitions
agg = cvs.raster(xarr)
```

```
plt.imshow(xarr.data)
```

```
agg.data
```

In [ ]:
#cvs.raster?

```
plt.imshow(agg.data)
```

```
plt.hist(elevation_xarray[0].data.flat, bins=100);
```

### Hillshade

```
austin_shade = hillshade(xr.DataArray(data_austin), out_type='data')
```

```
from datashader import transfer_functions as tf
img = tf.shade(austin_shade, cmap='gray', how='linear')
img
```

```
hv.Image(austin_shade.data, label='Austin Mountains').options(colorbar=True).options(cmap='gray')
```

```
austin_shade.stack
```

```
# Visualize the shade
cmap = plt.cm.terrain
rgba = plt.Normalize()(data_austin)
rgba = cmap(rgba)
rgba[..., -1] = 1 - 0.5*normalize(austin_shade.data)
plt.imshow(rgba);
```

## Landsat Red/NIR

```
# Open the elevetaion terrain file
#
import rasterio

# Landsat
landsat_red = 'assets/images/MERCATOR_LC80210392016114LGN00_B4.TIF'
landsat_nir = 'assets/images/MERCATOR_LC80210392016114LGN00_B5.TIF'

with rasterio.open(landsat_red) as fp:
    data_landsat_red = fp.read()

with rasterio.open(landsat_nir) as fp:
    data_landsat_nir = fp.read()

data_landsat_red = data_landsat_red[0]
data_landsat_nir = data_landsat_nir[0]
print('Image shapes:', data_landsat_red.shape, data_landsat_nir.shape)
```

```
img_red = hv.Image(data_landsat_red, label='Landsat Red filter').options(colorbar=True).options(cmap='Greens')
img_nir = hv.Image(data_landsat_nir, label='Landsat NIR filter').options(colorbar=True).options(cmap='Blues')
img_red + img_nir
```

```
# Let's cut a region in the middle to have a closer view of the scene
#
data_landsat_red = data_landsat_red[2000:5000, 2000:5000]
data_landsat_nir = data_landsat_nir[2000:5000, 2000:5000]
```

```
img_red = hv.Image(data_landsat_red, label='Landsat Red filter').options(colorbar=True).options(cmap='Greens')
img_nir = hv.Image(data_landsat_nir, label='Landsat NIR filter').options(colorbar=True).options(cmap='Blues')
img_red + img_nir
```

As we may notice, the images intensities span quite different range of values.
Let's apply a histogram equalization and normalize the values to have a clearer view of the scene.

```
from datashader.transfer_functions import eq_hist

data_landsat_red_eq = eq_hist(data_landsat_red)
data_landsat_nir_eq = eq_hist(data_landsat_nir)
````

```
img_red = hv.Image(data_landsat_red_eq, label='Landsat Red filter').options(colorbar=True).options(cmap='gray')
img_nir = hv.Image(data_landsat_nir_eq, label='Landsat NIR filter').options(colorbar=True).options(cmap='gray')
img_red + img_nir
```

```
data_landsat_ndvi = ndvi(xr.DataArray(data_landsat_nir_eq), xr.DataArray(data_landsat_red_eq))
```

```
hv.Image(data_landsat_ndvi.data).options(colorbar=True).options(cmap='PRGn')
```

The output of *NDVI* ranges from [-1,+1], where `-1` means more "Red" radiation while `+1` means more "NIR" radiation as observed by Landsat. A higher NIR radiation traces the presence of vegetation.